# 1 Create OD Matrix

The goal of the following notebook is to compute an OD-matrix using the preprocessed Milan Dataset.

The cells denoted with the symbol \* may be skipped if you want to use the pre-computed OD matrix

___

To compute the OD-Matrix M we first divide the urban environment into squared tiles of a given side. Second,
we use real mobility data (the Milan Dataset) to estimate the flows between the tiles, thus obtaining an origin-destination matrix $M$ where an element $m_{o,d} \in M$ describes the number of vehicles’ trips that start in tile $o$
and end in tile $d$.
___

In [ ]:
from utils import *
import geopandas as gpd
import pandas as pd
import sumolib
from skmob.tessellation import tilers
from skmob.utils import constants
import json
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

#### File paths

In [ ]:
# real mobility data path
mobility_data_path = "../data/preprocessed/MilanoData_big_preprocessed.csv"

# road network path
road_network_path = "../data/road_net/Milano_big/Milano_big_road_network.net.xml"

# shapefile path
shapefile_path = "../data/shapes/Milano_big_shape.geojson"

# outputs directories
od_matrix_path = "../data/OD_matrices"

#### Load the real mobility data *

In [ ]:
traj_D = pd.read_csv(mobility_data_path)
traj_D = skmob.TrajDataFrame(traj_D[['uid', 'datetime', 'lat', "lng"]], latitude='lat', longitude='lng', 
                                             user_id='uid', datetime='datetime')
traj_D[:4]

#### Load the shapefile of the geographic area of interest

In [ ]:
shape = gpd.GeoDataFrame.from_file(shapefile_path)

#### Create a squared tessellation of the city (size of 1km)

In [ ]:
tile_size_meters = 1000
tessellation_squared = tilers.tiler.get('squared', base_shape=shape, meters=tile_size_meters)
#tessellation = tilers.tiler.get('h3_tessellation', base_shape=shape, meters=tile_size_meters)

In [ ]:
tessellation_squared.shape

In [ ]:
#tessellation_squared.boundary.plot()
tessellation_squared.explore()

In [ ]:
tile_h3_resolution = constants.H3_UTILS['average_hexagon_edge_length']['8']
tessellation_hex = tilers.tiler.get('h3_tessellation', base_shape=shape, meters=tile_h3_resolution*1000)

In [ ]:
tessellation_hex.shape

In [ ]:
#tessellation_hex.boundary.plot()
tessellation_hex.explore()

#### Compute the OD matrix *

In [ ]:
od_matrix = compute_od_matrix(traj_D, tessellation_hex, traj_id="uid", self_loops=True)

#### Save the OD matrix *

It will be used later to compute the Mobility Demand

In [ ]:
with open(od_matrix_path+'/Milano3_OD_h3_8.npy', 'wb') as f:
    np.save(f, od_matrix)

#### Load the road network

In [ ]:
road_network = sumolib.net.readNet(road_network_path, withInternal=False)

In [ ]:
print('#Edges road net: '+str(len(road_network.getEdges())))

#### Assign road network edges to the corresponding tile

In [ ]:
dict_tile_edges = create_dict_tile_edges(road_network, tessellation_hex, exclude_roundabouts=True)

In [ ]:
flat_list = [item for sublist in list(dict_tile_edges.values()) for item in sublist]
print('#Edges assigned to tiles: '+str(len(flat_list)))

In [ ]:
print('#tiles: '+str(len(dict_tile_edges)))

In [ ]:
output_file = open(od_matrix_path+"/Milano3_tile_edges_h3_8.json", "w")
json.dump(dict_tile_edges, output_file)
output_file.close()